In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from subprocess import check_output

In [ ]:
dest = pd.read_csv("destinations.csv")
te = pd.read_csv("test.csv")
tr = pd.read_csv("train.csv")
tr["date_time"] = pd.to_datetime(tr["date_time"])
tr["year"] = tr["date_time"].dt.year
tr["month"] = tr["date_time"].dt.month

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
dest_pca = pca.fit_transform(dest[["d{0}".format(i + 1) for i in range(149)]])
dest_pca = pd.DataFrame(dest_pca)
dest_pca["srch_destination_id"] = dest["srch_destination_id"]
d1=tr
d2=te
df=d1
df["srch_ci"] = pd.to_datetime(df["srch_ci"], format='%Y-%m-%d', errors="coerce")
df["srch_co"] = pd.to_datetime(df["srch_co"], format='%Y-%m-%d', errors="coerce")
features = {}
for property in ["month", "day", "hour", "minute", "dayofweek", "quarter"]:
    features[property] = getattr(df["date_time"].dt, property)


c = [f for f in df.columns if f not in ["date_time", "srch_ci", "srch_co"]]
for property in c:
    features[property] = df[property]

In [ ]:
time_prop = ["month", "day", "dayofweek", "quarter"]
for property in time_prop:
    features["ci_{0}".format(property)] = getattr(df["srch_ci"].dt, property)
    features["co_{0}".format(property)] = getattr(df["srch_co"].dt, property)
features["stay_time"] = (df["srch_co"] - df["srch_ci"]).astype('timedelta64[h]')
r = pd.DataFrame(features)
r = r.join(dest_pca, on="srch_destination_id", how='left', rsuffix="dest")
r = r.drop("srch_destination_iddest", axis=1)
d1 = df

In [ ]:
def new_feature(items):
    return "_".join([str(i) for i in items])


match_f = ["srch_destination_id"]
clust = match_f + ['hotel_cluster']
groups = d1.groupby(clust)
result = {}
for user, grp in groups:
    clk = len(grp.is_booking[grp.is_booking == False])
    bookings = len(grp.is_booking[grp.is_booking == True])

    scores = bookings + .15 * clk

    clus_name = new_feature(user[:len(match_f)])
    if clus_name not in result:
        result[clus_name] = {}
    result[clus_name][user[-1]] = scores

In [ ]:
import operator

cluster_result = {}
for n in result:
    tw = result[n]
    tp = [l[0] for l in sorted(tw.items(), key=operator.itemgetter(1), reverse=True)[:5]]
    cluster_result[n] = tp

prediction = []
for t, users in d2.iterrows():
    hotel_cluster = new_feature([users[m] for m in match_f])
    if hotel_cluster in cluster_result:
        prediction.append(cluster_result[hotel_cluster])
    else:
        prediction.append([])


In [ ]:
match_f = ['user_location_country', 'user_location_region', 'user_location_city', 'hotel_market',
              'orig_destination_distance']

groups = d1.groupby(match_f)


def generate_match(row, match_cols):
    users = tuple([row[t] for t in match_cols])
    group = groups.get_group(users)
    
    hotel_clusters = list(set(group.hotel_cluster))
    return hotel_clusters


match_result = []
for i in range(d2.shape[0]):
    match_result.append(generate_match(d2.iloc[i], match_f))

In [ ]:
popular_hotel = list(tr.hotel_cluster.value_counts().head().index)
def f5(seq, idfun=None):
    if idfun is None:
        def idfun(x): return x
    seen = {}
    result = []
    for item in seq:
        marker = idfun(item)
        if marker in seen: continue
        seen[marker] = 1
        result.append(item)
    return result

In [ ]:
final_prediction = [f5(match_result[p] + prediction[p] + popular_hotel)[:5] for p in range(len(prediction))]

In [ ]:
from pandas.core.frame import DataFrame
final_preds=DataFrame(final_prediction)
final_preds.head()

final_preds.to_csv('predict_2.csv', header=True, index_label='id')

write_ = [" ".join([str(l) for l in p]) for p in final_prediction]

final_result=DataFrame(write_, columns=["hotel_cluster"])
final_result.head()

final_result.to_csv('result.csv', header=True, index_label='id')